In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
import pickle
soc = TopSoc()

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
def setAndRead(fTone, average=False, verbose=False, outsel="product", setDds=True):
    scan.setTones(np.array([fTone]), np.array([0.9]), np.array([0.0]))
    scan.prepRead(decimation=2, outsel=outsel, setDds=setDds)
    xs = scan.readAndUnpack(nt=1, nsamp=10000, iBegin=1000, average=average)
    if verbose:
        f = scan.toneFreqs[0]
        outCh = scan.outChs[0]
        outDds = scan.outDds[0]
        inCh = scan.inChs[0]
        inOffset = scan.inOffsets[0]
        ntran = scan.ntranByTone[0]
        stream = scan.streamByTone[0]
        print(" f=%8.3f  outCh=%3d  outDds=%+6.3f  inCh=%4d   inOffset=%+6.3f   ntran=%4d   stream=%1d"%(fTone, outCh, outDds, inCh, inOffset, ntran, stream))
    return xs


In [ ]:
fMixer = 614
soc.set_mixer(fMixer)
freqs = np.linspace(300,1000,250)

amps = np.zeros(len(freqs))
for i,freq in enumerate(freqs):
    print(i, end=" ")
    x = setAndRead(freq, verbose=True)
    amp = np.abs(x[0][0].mean())
    amps[i] = amp

In [ ]:
plt.plot(freqs,amps, '-o')
#plt.xlim((997, 998))

In [ ]:
inds = amps < 1000
print("bad freqs ",freqs[inds])

In [ ]:
raise Exception()

In [ ]:
freq = 446.34538153
x = setAndRead(freq, verbose=True, average=False)
amp = np.abs(x[0][0].mean())
print("amp =",amp)
      

In [ ]:
for inCh in range(soc.nInCh):
    fdds = .1
    soc.ddscic.set_ddsfreq(ch_id=inCh, f=fdds)
    if inCh == 193:
        print( "inCh, fdds", inCh, fdds)

In [ ]:
x2 = setAndRead(freq, verbose=True, average=False, outsel="dds", setDds=False)

In [ ]:
from numpy.fft import fft, fftshift
from scipy.signal import welch
xc = np.array(x2[0][0])
xc.shape
fs = soc.pfb_in.get_fb()/soc.ddscic.get_decimate()
F, Y = fftshift(welch(xc,
                          fs,
                          return_onesided=False,
                          detrend=False,
                          nperseg=512), axes=1)


In [ ]:
plt.plot((F),10*np.log10(Y))


In [ ]:
plt.plot(np.real(xc))
plt.plot(np.imag(xc))
plt.xlim((2000,2100))

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
fMixer = 614
freq = 1000.06
inCh,inOff = soc.inFreq2chOffset(freq)
print(inCh, inOff)
inFCenter = soc.inCh2FreqCenter(inCh)
print(inFCenter)
soc.set_mixer(fMixer)
x = setAndRead(freq, verbose=True, average=False, outsel="product")

In [ ]:
xc = np.array(x[0][0])
xc.shape
fs = soc.pfb_in.get_fb()/soc.ddscic.get_decimate()
F, Y = fftshift(welch(xc,
                          fs,
                          return_onesided=False,
                          detrend=False,
                          nperseg=512), axes=1)


In [ ]:
plt.plot((F)/1000,10*np.log10(Y))

In [ ]:
from numpy.fft import fft, fftshift

In [ ]:
#############################
### Channels Side-by-Side ###
#############################
# Quantization.
soc.pfb_in.qout(0)

# Set Decimation value.
soc.ddscic.decimation(value=2)

# Overwrite qsel.
qsel = soc.ddscic.qdata_qsel_reg
#soc.ddscic.qsel(20)

# By-pass DDS product.
soc.ddscic.dds_outsel(outsel="dds")

# Channel's sampling frequency.
fs = soc.pfb_in.get_fb()/soc.ddscic.get_decimate()

# Channels to plot.
#K = np.arange(soc.pfb_in.N)
#K = np.arange(250)
#K = [403,404,405,406]
K = np.arange(255,260)

plt.figure(dpi=150)

# Frequency and amplitude vectors.
FF = []
AA = []
for KK in K:
    print("Iteration K = %d" %KK)
    fck = KK*soc.pfb_in.get_fc()
    
    # Un-mask channel.
    soc.chsel.set_single(KK)
    
    # Transfer data.
    [xi,xq] = soc.stream.get_data(nt=1,idx=soc.chsel.ch2idx_orig(KK),debug=False)
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*fs
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog)

In [ ]:
soc.ddscic.dds_stop()

In [ ]:
soc.ddscic.dds_start()

In [ ]:
np.arange(200,220)

In [ ]:
soc.ddscic.qdata_qsel_reg